In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
# Example with tf.train.batch dynamic padding
# ==================================================

tf.reset_default_graph()

# Create a tensor [0, 1, 2, 3, 4 ,...]
x = tf.range(1, 51, name="x")

# A queue that outputs 0,1,2,3,...
range_q = tf.train.range_input_producer(limit=51, shuffle=True)
slice_end = range_q.dequeue()

# Slice x to variable length, i.e. [0], [0, 1], [0, 1, 2], ....
y = tf.slice(x, [0], [slice_end], name="y")

print(y)

# Batch the variable length tensor with dynamic padding
batched_data, batch_leng = tf.train.batch(
    tensors=[y, slice_end],
    batch_size=100,
    dynamic_pad=True,
    name="y_batch"
)

print(batched_data)
print(batch_leng) 

# Run the graph
# tf.contrib.learn takes care of starting the queues for us
res = tf.contrib.learn.run_n({"y": batched_data, 'len':batch_leng}, n=100, feed_dict=None)

# Print the result
print len(res)
print res[0]["y"]
print res[0]["len"]

inputs= np.array([batch_input["y"] for batch_input in res])
input_length= np.array([batch_input["len"] for batch_input in res])
outputs = np.array([batch_input["y"] for batch_input in res]) +1

mask = 1*(inputs>0)

print inputs.shape
print mask.shape
print input_length.shape
print outputs.shape

In [ ]:
from tensorflow.python.ops import rnn, rnn_cell
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 100
display_step = 10
epoch_per_step = 100

# Network Parameters
n_input = 1
n_steps = 50 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 50 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float32", [None, n_steps, n_input])
y = tf.placeholder("float32", [None, n_classes])


# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [ ]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps)
    # Required shape: 'n_steps' tensors list of shape (batch_size)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(0, n_steps, x)


    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = inputs[step%epoch_per_step],outputs[step%epoch_per_step] 
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"
